## Fuzzy String Matching
Fuzzy String Matching, also known as Approximate String Matching, is the process of finding strings that approximately match a pattern. 

In Python, Fuzzywuzzy is a library that uses Levenshtein Distance to calculate the differences between sequences and patterns. 

In [11]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import pandas as pd

In [16]:
df_dbpedia = pd.read_csv("/Users/twei/workplace/ISE-workplace/ISE-Linking-Entities-from-Images-to-Knowledge-Graphs/src/data/dbpedia_classes.csv")
df_imagenet = pd.read_csv("/Users/twei/workplace/ISE-workplace/ISE-Linking-Entities-from-Images-to-Knowledge-Graphs/src/data/dbpedia_classes.csv")

,class,label
0,http://dbpedia.org/ontology/Company,company
1,http://dbpedia.org/ontology/Activity,activity
2,http://dbpedia.org/ontology/Name,name
3,http://dbpedia.org/ontology/Person,person
4,http://dbpedia.org/ontology/Actor,actor
...,...,...
778,http://dbpedia.org/ontology/WindMotor,Wind motor
779,http://dbpedia.org/ontology/Windmill,Windmill
780,http://dbpedia.org/ontology/Wine,wine
781,http://dbpedia.org/ontology/Woman,woman
